
# K-Medoids clustering algorithm


#### Annisa Gita Asmara 
14116027




In [48]:
        import numpy

        from pyclustering.cluster.encoder import type_encoding
        from pyclustering.utils import read_sample,medoid
        from pyclustering.samples.definitions import FCPS_SAMPLES
        from pyclustering.core.metric_wrapper import metric_wrapper
        from pyclustering.utils.metric import distance_metric, type_metric
        from pyclustering.core.wrapper import ccore_library
        from pyclustering.cluster.kmedoids import kmedoids
        from pyclustering.cluster import cluster_visualizer
        from matplotlib.colors import LogNorm
        from sklearn import datasets
        from sklearn.decomposition import PCA
        from sklearn.externals import joblib
        from sklearn.cluster import AgglomerativeClustering, DBSCAN, KMeans
        from sklearn.metrics.pairwise import pairwise_distances
        from sklearn.mixture import GaussianMixture
        from sklearn.preprocessing import LabelEncoder
        from sklearn import metrics
        from sklearn.datasets.samples_generator import make_blobs
        from sklearn.preprocessing import StandardScaler

        import matplotlib.pyplot as plt
        import numpy as np
        import pandas as pd
        import pylab as pl
        import pyclustering.core.kmedoids_wrapper as wrapper


In [49]:

        sample = read_sample(FCPS_SAMPLES.SAMPLE_TWO_DIAMONDS)
 
        initial_medoids = [1, 500]
 
        kmedoids_instance = kmedoids(sample, initial_medoids)
 
        kmedoids_instance.process()
        clusters = kmedoids_instance.get_clusters()
 
        # Show allocated clusters.
        print(clusters)
        
        # Display clusters.
        visualizer = cluster_visualizer()
        visualizer.append_clusters(clusters, sample)
        visualizer.show()

        # create Minkowski distance metric with degree equals to '2'
        metric = distance_metric(type_metric.MINKOWSKI, degree=2)
 
         # create K-Medoids algorithm with specific distance metric
        kmedoids_instance = kmedoids(sample, initial_medoids, metric=metric)
 
         # run cluster analysis and obtain results
        kmedoids_instance.process()
        clusters = kmedoids_instance.get_clusters()
        
        # calculate distance matrix for sample
        sample = read_sample(path_to_sample)
        matrix = calculate_distance_matrix(sample)
 
        # create K-Medoids algorithm for processing distance matrix instead of points
        kmedoids_instance = kmedoids(matrix, initial_medoids, data_type='distance_matrix')
 
        # run cluster analysis and obtain results
        kmedoids_instance.process()
 
        clusters = kmedoids_instance.get_clusters()
        medoids = kmedoids_instance.get_medoids()

IndentationError: unexpected indent (<ipython-input-49-17b74b938f29>, line 2)

In [50]:

        def __init__(self, data, initial_index_medoids, tolerance=0.001, ccore=True, **kwargs):
        self.__pointer_data = data
        self.__clusters = []
        self.__medoid_indexes = initial_index_medoids
        self.__tolerance = tolerance
 
        self.__metric = kwargs.get('metric', distance_metric(type_metric.EUCLIDEAN_SQUARE))
        self.__data_type = kwargs.get('data_type', 'points')
        self.__itermax = kwargs.get('itermax', 200)
 
        self.__distance_calculator = self.__create_distance_calculator()
 
        self.__ccore = ccore and self.__metric.get_type() != type_metric.USER_DEFINED
        if self.__ccore:
            self.__ccore = ccore_library.workable()
 
 
    def process(self):

         
        if self.__ccore is True:
            ccore_metric = metric_wrapper.create_instance(self.__metric)
            self.__clusters, self.__medoid_indexes = wrapper.kmedoids(self.__pointer_data, self.__medoid_indexes, self.__tolerance, self.__itermax, ccore_metric.get_pointer(), self.__data_type)
         
        else:
            changes = float('inf')
            iterations = 0
 
            while changes > self.__tolerance and iterations < self.__itermax:
                self.__clusters = self.__update_clusters()
                update_medoid_indexes = self.__update_medoids()
 
                changes = max([self.__distance_calculator(self.__medoid_indexes[index], update_medoid_indexes[index]) for index in range(len(update_medoid_indexes))])
 
                self.__medoid_indexes = update_medoid_indexes
 
                iterations += 1
 
        return self
 
 
    def get_clusters(self):
        return self.__clusters
     
     
    def get_medoids(self):
 
        return self.__medoid_indexes
 
 
    def get_cluster_encoding(self):
         
        return type_encoding.CLUSTER_INDEX_LIST_SEPARATION
 
 
    def __create_distance_calculator(self):
        
        if self.__data_type == 'points':
            return lambda index1, index2: self.__metric(self.__pointer_data[index1], self.__pointer_data[index2])
 
        elif self.__data_type == 'distance_matrix':
            if isinstance(self.__pointer_data, numpy.matrix):
                return lambda index1, index2: self.__pointer_data.item((index1, index2))
 
            return lambda index1, index2: self.__pointer_data[index1][index2]
 
        else:
            raise TypeError("Unknown type of data is specified '%s'" % self.__data_type)
 
 
    def __update_clusters(self):
        clusters = [[self.__medoid_indexes[i]] for i in range(len(self.__medoid_indexes))]
        for index_point in range(len(self.__pointer_data)):
            if index_point in self.__medoid_indexes:
                continue
 
            index_optim = -1
            dist_optim = float('Inf')
             
            for index in range(len(self.__medoid_indexes)):
                dist = self.__distance_calculator(index_point, self.__medoid_indexes[index])
                 
                if dist < dist_optim:
                    index_optim = index
                    dist_optim = dist
             
            clusters[index_optim].append(index_point)
         
        return clusters
     
     
    def __update_medoids(self):
 
        medoid_indexes = [-1] * len(self.__clusters)
         
        for index in range(len(self.__clusters)):
            medoid_index = medoid(self.__pointer_data, self.__clusters[index], metric=self.__metric, data_type=self.__data_type)
            medoid_indexes[index] = medoid_index
              
        return medoid_indexes

IndentationError: unexpected indent (<ipython-input-50-d7816c7b162f>, line 2)